In [ ]:
from lecode_util.call_llm import ask_tyqw,ask_llm

keyfact_Evaluation_PROMPT_cot = """分析用户-律师咨询对话中提及的关键信息，并将其与给定的关键信息列表进行匹配。
输入数据：
1. 用户-律师对话 (user_lawyer_dialogue):
   - 格式为：{对话ID: {'q': 问题, 'a': 回答}}
   - 每组对话包含律师提问和用户回答

2. 关键信息列表 (key_information_li):
   - 格式为：{序号: 信息内容}
   - 序号从0开始

任务要求：
1. 识别对话中出现的关键信息
2. 将识别到的信息与关键信息列表中的条目进行匹配
3. 记录匹配信息的序号

输出格式要求：
- 输出应为一个 Python 字典
- 键：对话ID (如 "QA0", "QA1"...)
- 值：该对话中提及的关键信息序号列表
- 如果没有任何关键信息被提及，则返回一个空字典{}。
- 示例格式：
```python
{"QA0":[0,1],"QA1":2,...}
```

以下是咨询对话和关键信息列表:
[咨询对话]
{user_lawyer_dialogue}

[关键信息列表]
{key_information_li}

Let's think step by step:
"""
keyfact_Evaluation_PROMPT_nocot = """分析用户-律师咨询对话中提及的关键信息，并将其与给定的关键信息列表进行匹配。
输入数据：
1. 用户-律师对话 (user_lawyer_dialogue):
   - 格式为：{对话ID: {'q': 问题, 'a': 回答}}
   - 每组对话包含律师提问和用户回答

2. 关键信息列表 (key_information_li):
   - 格式为：{序号: 信息内容}
   - 序号从0开始

任务要求：
1. 识别对话中出现的关键信息
2. 将识别到的信息与关键信息列表中的条目进行匹配
3. 记录匹配信息的序号

输出格式要求：
- 输出应为一个 Python 字典
- 键：对话ID (如 "QA0", "QA1"...)
- 值：该对话中提及的关键信息序号列表
- 如果没有任何关键信息被提及，则返回一个空字典{}。
- 示例格式：
```python
{"QA0":[0,1],"QA1":2,...}
```

以下是咨询对话和关键信息列表:
[咨询对话]
{user_lawyer_dialogue}

[关键信息列表]
{key_information_li}

- 仅输出结果，不要有任何思考过程
"""

suggestion_evaluation_prompt = """你是一位经验丰富的法律专家，专门评估法律咨询对话的法律建议的质量。请以公正、严谨的评判者身份，对AI助手提供的法律建议的质量进行客观评估。在评估时，请从以下五个关键维度进行分析和评分：

1.专业性
- 准确理解客户的问题并提供相关答案
- 清晰简洁地解释复杂的法律知识
- 建议的可操作性
2.流畅性
- 确保语义连贯性，没有逻辑错误或不相关的信息
- 保持风格和内容的一致性
- 保持友好、热情的回答态度
3.完整性
- 提供了足够的信息和细节
- 没有遗漏重要的建议
4.服务性
- 回应的针对性，提供个性化的解决方案
- 使用平易近人的表达方式，语言的亲和度
- 注重客户体验和感受,对当事人诉求的共情与尊重
5.安全性
- 提供科学、准确的法律知识。
- 确保客户安全;拒绝可能造成伤害的信息或建议。
- 遵守职业道德并尊重客户的选择。不得包含歧视性、偏见性或有害内容。

我们将提供以下材料: 高质量的参考答案, AI助手撰写的法律建议。
在开始评估时，你需要遵循以下推理步骤：
1. 比较AI助手撰写的法律建议与参考答案，指出AI助手答案中的不足之处，并详细解释。 
2. 根据上述维度评估AI助手的法律建议，每个维度给出1到10分的分数。
3. 基于各维度的得分，得出AI助手的综合评分（1-10分）。
4. 你的评分应尽可能严格，并且必须遵循以下评分规则：回复的质量越高，得分越高。
 
评分标准：
1-2分：建议包含与案件无关内容，存在严重错误，包含未经验证或虚假信息，或产生具有潜在危害的内容。
3-4分：建议无重大错误，但在法律关系界定或关键点回应上存在明显不足，逻辑推理较差，缺乏针对性,建议过于笼统，未达到基本咨询要求。
5-6分：建议基本符合咨询要求，法律分析准确但逻辑完整性一般，基本回应了咨询要点但缺乏深度分析，整体表现中规中矩。
7-8分：建议质量接近参考答案，供了切实可行的解决方案，各评估维度表现优秀，无明显缺陷。
9-10分：建议质量显著超越参考答案，各维度表现近乎完美，提供了极具价值的解决方案。
  
  
评分时请提供详细的评估注释。在每个维度分数后，请务必提供解释。所有分数都应为整数。最后，以以下字典格式返回评估结果：
 
```python{{'专业性': 分数, '流畅性': 分数, '完整性': 分数, '服务性': 分数, '安全性': 分数, '综合得分': 总分}}```
 
<法律咨询对话>

{dialogue}

</法律咨询对话>

<参考答案>
 
{gt_suggestion}
 
</参考答案>

<AI Assistant待评估建议>
 
{pred_suggestion}
 
</AI Assistant待评估建议>
 
请开始评估：
"""


import re
# 使用正则表达式提取内容
def extract_python(text):
    if "```python" in text:
        pattern = r'```python\n(.*?)\n```'

        match = re.search(pattern, text, re.DOTALL)

        if match:
            extracted_content = match.group(1)

            return True,eval(extracted_content)
        else:
            return  False,None
    else:
        try:
            return True,eval(text)
        except:
            return  False,None
def count_key_fact_num(llm_keyfact_dict):
    if llm_keyfact_dict==str:
        llm_keyfact_dict =eval(llm_keyfact_dict)
    if len(llm_keyfact_dict) ==0:
        return 0
    
    else:
        res_ = []
        for k in llm_keyfact_dict:
            res_.extend(llm_keyfact_dict[k])
        return len(set(res_))
def compute_recall5(num):
    try:
        if type(df['llm_evaluate_keyfact'][num]) ==str:
            res_dict = eval(df['llm_evaluate_keyfact'][num])
        else:
            res_dict = df['llm_evaluate_keyfact'][num]
        res_set = []
        for k in res_dict:
            if k <5:
                res_set.extend(res_dict[k])
        res_set = set(res_set) 
        # importance = dict((int(k), v) for k, v in eval(df['gt_key_fact_importance'][num]).items())
        len_key = len(eval(df['gt_key_fact_importance'][num]))


        return len(res_set)/len_key
    except:
        
        return None
def compute_weighted_recall(num):
    try:
        # res_dict = eval(df['llm_evaluate_keyfact'][num])
        if type(df['llm_evaluate_keyfact'][num]) ==str:
            res_dict = eval(df['llm_evaluate_keyfact'][num])
        else:
            res_dict = df['llm_evaluate_keyfact'][num]
        res_set = []
        for k in res_dict:
            res_set.extend(res_dict[k])
        res_set = set(res_set) 
        importance = dict((int(k), v) for k, v in eval(df['gt_key_fact_importance'][num]).items())

        all_weight = sum(importance.values())
        recall_weight = sum([importance[i] for i in res_set])

        return recall_weight/all_weight
    except:
        
        return None
import math
def compute_ndcg(num):
    # 用于记录 key fact 的id 是否出现过，只统计最先出现的
    try:
        existed_li = []
        if type(df['llm_evaluate_keyfact'][num]) ==str:
            res_dict = eval(df['llm_evaluate_keyfact'][num])
        else:
            res_dict = df['llm_evaluate_keyfact'][num]
        # print(res_dict)
        importance = dict((int(k), v) for k, v in eval(df['gt_key_fact_importance'][num]).items())
        # print(importance)
        importance_dict = {}
        for query_id in res_dict:
            importance_v = 0
            for val in res_dict[query_id]: # 循环每一个key fact id （val）

                if val not in existed_li:
                    existed_li.append(val) # 将统计过了的放入existedli
                    importance_v = importance_v+importance[val] #--> 对应 key fact的相关性
                int(query_id)+1 # --> 对应的位置
            importance_dict[int(query_id)+1] = importance_v
        DCG = 0
        # print(importance_dict)
        for k,v in importance_dict.items():
            DCG = DCG + v/math.log(k+1,2)
        # print(DCG)
        iDCG = 0
        for idx, v in enumerate(sorted(list(importance_dict.values()),reverse=True)):
            iDCG = iDCG+ v/math.log(idx+1+1,2)
        if iDCG == 0:
            return 0
        else:
            return DCG/iDCG
    except:
        
        return None
    
def process_dialogue(num,model_name='qwen-max'):
    pred_diag = eval(df['pred_dialogue'][num])[1:-1]
    pred_diag = [[i[0],i[1].replace("提问:","").replace("用户:","").replace("不知道。","不知道")] for i in pred_diag]


    # 在转换时记录原始索引
    qa_pairs = []
    for i in range(0, len(pred_diag), 2):
        if i+1 < len(pred_diag):  # 确保有问答对
            qa_pair = {
                'q': pred_diag[i][1],      # 律师的问题
                'a': pred_diag[i+1][1],    # 用户的回答
                'original_idx': i//2        # 记录原始索引位置
            }
            qa_pairs.append(qa_pair)

    # 过滤掉"不知道"的回答并保持原始索引
    filtered_qa = [qa for qa in qa_pairs if qa['a'] != '不知道']

    # 转换为numbered字典
    numbered_qa = {f"QA{i}": qa for i, qa in enumerate(filtered_qa)}


    user_lawyer_dialogue = {}
    for idx, qa in numbered_qa.items():
        user_lawyer_dialogue[idx] = {
            '律师': qa['q'],
            '用户': qa['a']
        }
        
    gt_key_fact = eval(df['gt_key_fact'][num])
    gt_key_fact = dict((int(k), v) for k, v in gt_key_fact.items())

    response, usage = ask_llm(keyfact_Evaluation_PROMPT_nocot.replace("{user_lawyer_dialogue}",str(user_lawyer_dialogue)).replace("{key_information_li}",str(gt_key_fact)),model=model_name)
    
    flag,result = extract_python(response)
    
    final_res = {}
    for key in result:
        final_res[numbered_qa[key]["original_idx"]] = result[key]
        
    if flag:
        return final_res
    else:
        return False
    

def process_suggestion(num,model_name='gpt-4o-mini'):

    response, usage = ask_llm(suggestion_evaluation_prompt.replace("{gt_suggestion}",df['gt_suggestion'][num]).replace("{pred_suggestion}",df['pred_suggestion'][num]).replace("{dialogue}",df['pred_dialogue'][num])
                              ,model=model_name)
        
    return response

    
import re
import ast

def check_suggestion_format(text):
    # 首先检查是否包含```python{...}```格式
    pattern = r'```python\s*({[^}]+})\s*```'
    match = re.search(pattern, text)
    if not match:
        return False
    
    try:
        # 提取字典字符串并转换为Python对象
        dict_str = match.group(1)
        dict_obj = ast.literal_eval(dict_str)
        
        # 检查是否是字典类型
        if not isinstance(dict_obj, dict):
            return False
        
        # 必需的键
        required_keys = {'专业性', '流畅性', '完整性', '服务性', '安全性', '综合得分'}
        
        # 检查键是否完全匹配
        if set(dict_obj.keys()) != required_keys:
            return False
        
        # 检查所有值是否都是1-10的整数
        for value in dict_obj.values():
            if not isinstance(value, int) or value < 1 or value > 10:
                return False
        
        return True
        
    except (ValueError, SyntaxError):
        return False
def extract_python_2(text):
    if "```python" in text:
        pattern = r'```python(.*?)```'

        match = re.search(pattern, text, re.DOTALL)

        if match:
            extracted_content = match.group(1)

            return True,eval(extracted_content)
        else:
            return  False,None
    else:
        try:
            return True,eval(text)
        except:
            return  False,None

def extract_suggestion_rate(text):
    _,score_dict = extract_python_2(text)
    return score_dict


In [2]:
import pandas as pd

path ="output_file_ask_q"


df = pd.read_csv(f"outputs/evaluation_output/{path}.csv")


mode_type="concurrent" # 选择并行来评估
# mode_type="sequence" # 选择一条一条数据来评估

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

if mode_type =="sequence":
    final_res = []
    error_li = []
    for num in tqdm(range(0,len(df))):
        res = process_dialogue(num)
        if res !=False:
            final_res.append(res)
        else:
            final_res.append(None)
            error_li.append(num)
elif mode_type=="concurrent":
    num_workers = 32*3  # 可以根据需求调整

    final_res = [None] * len(df)  # 预分配列表
    error_li = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # 提交所有任务
        future_to_idx = {
            executor.submit(process_dialogue, i): i 
            for i in range(len(df))
        }

        # 使用tqdm显示进度
        for future in tqdm(
            as_completed(future_to_idx),  # 使用 as_completed 而不是 concurrent.futures.as_completed
            total=len(df),
            desc="Processing"
        ):
            idx = future_to_idx[future]
            try:
                result = future.result()
                if result != False:
                    final_res[idx] = result
                else:
                    final_res[idx] = None
                    error_li.append(idx)
            except Exception as e:
                print(f"Error processing index {idx}: {str(e)}")
                error_li.append(idx)

   

In [5]:
error_li = [idx for idx,i in enumerate(final_res) if i==None]

In [6]:
error_li_1 =[]
with ThreadPoolExecutor(max_workers=num_workers) as executor:
    # 提交所有任务
    future_to_idx = {
        executor.submit(process_dialogue, i): i 
        for i in error_li
    }

    # 使用tqdm显示进度
    for future in tqdm(
        as_completed(future_to_idx),  # 使用 as_completed 而不是 concurrent.futures.as_completed
        total=len(error_li),
        desc="Processing"
    ):
        idx = future_to_idx[future]
        try:
            result = future.result()
            if result != False:
                final_res[idx] = result
            else:
                final_res[idx] = None
                error_li_1.append(idx)
        except Exception as e:
            print(f"Error processing index {idx}: {str(e)}")
            error_li_1.append(idx)

Processing: 100%|██████████| 6/6 [00:10<00:00,  1.73s/it]


In [ ]:
df["llm_evaluate_keyfact"] = final_res

df["llm_evaluate_keyfact_num"] = df["llm_evaluate_keyfact"].apply(count_key_fact_num)
# df
df["llm_evaluate_keyfact_recall"] = df["llm_evaluate_keyfact_num"]/df['gt_key_fact'].apply(eval).apply(len)
df["llm_evaluate_keyfact_precision"] = df["llm_evaluate_keyfact_num"]/(df['pred_dialogue'].apply(eval).apply(len)-2)/2

# 计算recall at 5
recall5 = []
for num in range(0,len(df)):
    recall5.append(compute_recall5(num))
df['recall@5'] = recall5
#计算weighted recall
weighted_recall = []
for num in range(0,len(df)):
    weighted_recall.append(compute_weighted_recall(num))
df['weighted_recall'] = weighted_recall
# 计算ndcg
ndcg_li = []
for num in range(0,len(df)):
    ndcg_li.append(compute_ndcg(num))
df['ndcg'] = ndcg_li

In [1]:


from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

if mode_type =="sequence":
    final_res_suggestion = []
    error_li_suggestion = []
    for num in tqdm(range(0,len(df))):
        res = process_suggestion(num)

        final_res_suggestion.append(res)

elif mode_type=="concurrent":
    # 可以设置更多的线程数，因为IO操作不受GIL限制
    # num_workers = 32*3  # 可以根据需求调整
    num_workers=45
    final_res_suggestion = [None] * len(df)  # 预分配列表
    error_li_suggestion = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # 提交所有任务
        future_to_idx = {
            executor.submit(process_suggestion, i): i 
            for i in range(len(df))
        }

        # 使用tqdm显示进度
        for future in tqdm(
            as_completed(future_to_idx),  # 使用 as_completed 而不是 concurrent.futures.as_completed
            total=len(df),
            desc="Processing"
        ):
            idx = future_to_idx[future]
            try:
                result = future.result()
                if result != False:
                    final_res_suggestion[idx] = result
                else:
                    final_res_suggestion[idx] = None
                    error_li_suggestion.append(idx)
            except Exception as e:
                print(f"Error processing index {idx}: {str(e)}")
                error_li_suggestion.append(idx)



In [14]:
error_li_suggestion = []
for idx,i in enumerate(final_res_suggestion):
    if i!=None:
        if not check_suggestion_format(i):
            error_li_suggestion.append(idx)
    if i==None:
        error_li_suggestion.append(idx)
        
error_li_suggestion_1 =[]
with ThreadPoolExecutor(max_workers=num_workers) as executor:
    # 提交所有任务
    future_to_idx = {
        executor.submit(process_suggestion, i): i 
        for i in error_li_suggestion
    }

    # 使用tqdm显示进度
    for future in tqdm(
        as_completed(future_to_idx),  # 使用 as_completed 而不是 concurrent.futures.as_completed
        total=len(error_li_suggestion),
        desc="Processing"
    ):
        idx = future_to_idx[future]
        try:
            result = future.result()
            if result != False:
                final_res_suggestion[idx] = result
            else:
                final_res_suggestion[idx] = None
                error_li_suggestion_1.append(idx)
        except Exception as e:
            print(f"Error processing index {idx}: {str(e)}")
            error_li_suggestion_1.append(idx)

Processing: 100%|██████████| 23/23 [00:56<00:00,  2.44s/it]


In [15]:

error_li_suggestion = []
for idx,i in enumerate(final_res_suggestion):
    if i!=None:
        if not check_suggestion_format(i):
            error_li_suggestion.append(idx)
    if i==None:
        error_li_suggestion.append(idx)
        
print(len(error_li_suggestion))


0


In [16]:
df['suggestion_rate'] = final_res_suggestion
df['suggestion_rate_dict'] = df['suggestion_rate'].apply(extract_suggestion_rate)
df['专业性'] = df['suggestion_rate_dict'].apply(lambda x: x['专业性'])
df['流畅性'] = df['suggestion_rate_dict'].apply(lambda x: x['流畅性'])
df['完整性'] = df['suggestion_rate_dict'].apply(lambda x: x['完整性'])
df['服务性'] = df['suggestion_rate_dict'].apply(lambda x: x['服务性'])
df['安全性'] = df['suggestion_rate_dict'].apply(lambda x: x['安全性'])
df['综合得分'] = df['suggestion_rate_dict'].apply(lambda x: x['综合得分'])

In [17]:
df.to_csv(f"outputs/evaluation_output/{path}_evalgpt42.csv",index=False)

In [18]:
metrics = {
    "llm_evaluate_keyfact_recall": df['llm_evaluate_keyfact_recall'].mean(),
    "weighted_recall": df['weighted_recall'].mean(),
    "recall@5": df['recall@5'].mean(), 
    "ndcg": df['ndcg'].mean(),
    "ask_turn": df['ask_turn'].mean(),
    "dont_know_num": df['dont_know_num'].mean(),
    "rouge": df['rouge'].mean(),
    "bertscore": df['bertscore'].mean()
}

# 打印成表格形式
print("\nMetrics Summary:")
print("-" * 50)
for metric, value in metrics.items():
    print(f"{metric}\t{value:.4f}")
print("-" * 50)

# 打印成一行,方便复制到Excel
print("\nExcel Format:")
print("\t".join([f"{value:.4f}" for value in metrics.values()]))



Metrics Summary:
--------------------------------------------------
llm_evaluate_keyfact_recall	0.5380
weighted_recall	0.5508
recall@5	0.4345
ndcg	0.8472
ask_turn	20.3649
dont_know_num	0.4705
rouge	0.1848
bertscore	0.6748
--------------------------------------------------

Excel Format:
0.5380	0.5508	0.4345	0.8472	20.3649	0.4705	0.1848	0.6748


In [19]:
for key in df.columns[-6:]:
    print(key,df[key].mean())

专业性 5.922972972972973
流畅性 7.302702702702702
完整性 5.2337837837837835
服务性 5.5864864864864865
安全性 7.120270270270271
综合得分 5.944594594594594


# See all results

In [2]:
import os
import pandas as pd
path_li = [i for i in os.listdir("./outputs/evaluation_output/") if "_evalgpt42" in i]
# path_li

res_ = []
for p in path_li:
    df = pd.read_csv(f"./outputs/evaluation_output/{p}")
    
    res_.append([p.replace("output_file_","").replace("_eval.csv",""),
df['llm_evaluate_keyfact_recall'].mean(),df['weighted_recall'].mean(),
    df['recall@5'].mean(), df['ndcg'].mean(),df['ask_turn'].mean(),df['dont_know_num'].mean(),
    df['rouge'].mean(),df['bertscore'].mean(),
    df['专业性'].mean(), df['流畅性'].mean(), df['完整性'].mean(), df['服务性'].mean(), df['安全性'].mean(), df['综合得分'].mean()          ])
